# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

To import the json, navigate to the Resources folder within your terminal and use the following code:

mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json

In [45]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [46]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [47]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'classDB', 'config', 'epa', 'fruits_db', 'local', 'met', 'petsitly_marketing', 'travel_db', 'uk_food']


In [48]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [49]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [50]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': 'West Cottage',
 'AddressLine2': 'South Foreland Lighthouse',
 'AddressLine3': 'The Front',
 'AddressLine4': 'St Margarets Bay',
 'BusinessName': 'Mrs Knotts Tea Room',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.492467848017,
 'FHRSID': 429991,
 'LocalAuthorityBusinessID': 'PI/000055200',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6HP',
 'RatingDate': '2019-05-02T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('658f1d1ea082f8c24534c004'),
 'geocode': {'latitude': '51.14061', 'longitude': '1.371894'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/429991',
            'rel': 'self'}],
 'meta': {'dataSource': None,
  

In [51]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [52]:
# Create a dictionary for the new restaurant data
new_data = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [53]:
# Insert the new restaurant into the collection
establishments.insert_one(new_data)

InsertOneResult(ObjectId('658f1d2d1dd13cfc9614858a'), acknowledged=True)

In [54]:
# Check that the new restaurant was inserted
query = {"BusinessName": "Penang Flavours"}
print(establishments.count_documents(query))

1


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [55]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType': 'Restaurant/Cafe/Canteen'}
fields = {'BusinessTypeID': 1}
result = establishments.find_one(query, fields)
pprint(result)

{'BusinessTypeID': 1, '_id': ObjectId('658f1d1ea082f8c24534c004')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [56]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one({'BusinessName': 'Penang Flavours'}, [ {'$set': {'BusinessTypeID': 1}}])

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [57]:
# Confirm that the new restaurant was updated
query = {"BusinessName": "Penang Flavours"}
result = establishments.find_one(query)
pprint(result)

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('658f1d2d1dd13cfc9614858a'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [58]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {'LocalAuthorityName': 'Dover'}
print('The number of documents with the authority name = Dover: ', establishments.count_documents(query))

The number of documents with the authority name = Dover:  994


In [59]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many({'LocalAuthorityName': 'Dover'})

DeleteResult({'n': 994, 'ok': 1.0}, acknowledged=True)

In [60]:
# Check if any remaining documents include Dover
print('The number of documents with the authority name = Dover: ', establishments.count_documents(query))

The number of documents with the authority name = Dover:  0


In [61]:
# Check that other documents remain with 'find_one'
pprint(establishments.find_one())

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('658f1d1ea082f8c24534c2eb'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': 

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [62]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({}, [ {'$set': {'geocode.latitude': {'$toDouble': '$geocode.latitude'},
                                        'geocode.longitude': {'$toDouble': '$geocode.longitude'}
                                    }
                                }]
                            )

UpdateResult({'n': 38786, 'nModified': 38786, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

Use `update_many` to convert `RatingValue` to integer numbers.

In [63]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [64]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({}, [ {'$set': {'RatingValue': {'$toInt': '$RatingValue'}}}])

UpdateResult({'n': 38786, 'nModified': 34695, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [68]:
# Check that the coordinates and rating value are now numbers
query = {}
fields = {'geocode.latitude': 1, 'geocode.longitude': 1, 'RatingValue': 1}

result = establishments.find_one(query, fields)

pprint(result)

{'RatingValue': 5,
 '_id': ObjectId('658f1d1ea082f8c24534c2eb'),
 'geocode': {'latitude': 51.083812, 'longitude': 1.195625}}
